# Congressional Speech Analysis

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
pd.options.display.max_columns = 1200  
pd.options.display.max_rows = 1200 

import unicodedata
import itertools
import datetime
import datefinder
import operator
import string
import re

import nltk
from nltk.tokenize import sent_tokenize

def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))

### Get files

The first step is to generate the data frame of all 114th Congress hearings. The files are contained in directories following this structure:

Data ---> Congress ---> Office ---> Department ---> Text files in txt format.

Each text file is imported into the documents column of the data frame and assigned a document ID.

In [2]:
from IPython.display import clear_output

FILES_PATH = '/Users/yc00027/Documents/GitHub/congress_speech_analysis/Data'

congresses = sorted(listdir_nohidden(FILES_PATH))

congressional_hearings = []

for congress in congresses:
    clear_output()
    print(f'Now loading congress {congress}')
    offices = listdir_nohidden(congress)
    for office in offices:
        clear_output()
        print(f'Now loading office {office}')
        congress = os.path.split(os.path.dirname(office))[1]
        departments = listdir_nohidden(office)
        for department in departments:
            clear_output()
            office = os.path.split(os.path.dirname(department))[1]
            try:
                session_files = listdir_nohidden(department)
            except:
                print(f'No txt files for session {session}')
                continue

            txt_files = [f for f in session_files if f.endswith('.txt')]
            for file in txt_files:
                with open(file, errors='replace') as f:
                    document = f.read().lower() 
                    department = os.path.split(os.path.dirname(file))[1]
                    name = os.path.basename(file)
                    congressional_hearings.append([document, congress, office, department, name])
            
clear_output()
print('Done')

congressional_hearings_df = pd.DataFrame(congressional_hearings).reset_index().rename(columns={'index': 'document_id'})
congressional_hearings_df.columns = ['document_id', 'document', 'congress','office','department','name']
congressional_hearings_df = congressional_hearings_df.sort_values(by=['congress', 'office','department','name'])

print(f'Congressional Hearings DF shape: {congressional_hearings_df.shape}')
congressional_hearings_df.head()

Done
Congressional Hearings DF shape: (2535, 6)


,document_id,document,congress,office,department,name
1837,1837,\n - eastern mediterranean energy: challenges ...,114th,House,Ad Hoc Committee on Energy,1.txt
1659,1659,\n - [errata] manipulation and fraud in the re...,114th,House,Commission on Security and Cooperation in Europe,1.txt
1658,1658,\n - the rule of law and civil society in azer...,114th,House,Commission on Security and Cooperation in Europe,2.txt
1657,1657,\n - human rights violations in russian-occupi...,114th,House,Commission on Security and Cooperation in Europe,3.txt
1656,1656,\n - nato's warsaw summit and the future of eu...,114th,House,Commission on Security and Cooperation in Europe,4.txt


Inspect the data frame of bills for the 114th congress and extract full list of speaker names, number of bills and bill types.

In [3]:
bills93_114_df = pd.read_csv('bills93-114.csv', error_bad_lines=False, encoding='ISO-8859-1', sep = ';')
bills_114 = bills93_114_df[bills93_114_df['Cong'] == 114]

print(f'There were {bills_114.shape[0]} bills in Congress 114th.')

speaker_names = sorted(bills_114['NameFull'].str.lower().unique())
print(f'There were {len(speaker_names)} speakers in Congress 114th.')
print('')

print('Number of bills according to bill type:')
print(bills_114['BillType'].value_counts())
print('')

print('Bill types: "hr" (House Bill); "s" (Senate Bill); "hres" (House Resolution); "sres" (Senate Resolution); \
                   "hcon" (House Concurrent Resolution); "scon" (Senate Concurrent Resolution); \
                   "hjres" (House Joint Resolution); "sjres" (Senate Joint Resolution).')


There were 12043 bills in Congress 114th.
There were 542 speakers in Congress 114th.

Number of bills according to bill type:
hr         6508
s          3547
hres        956
sres        642
hconres     183
hjres       108
sconres      58
sjres        41
Name: BillType, dtype: int64

Bill types: "hr" (House Bill); "s" (Senate Bill); "hres" (House Resolution); "sres" (Senate Resolution);                    "hcon" (House Concurrent Resolution); "scon" (Senate Concurrent Resolution);                    "hjres" (House Joint Resolution); "sjres" (Senate Joint Resolution).


Inspect the data frame of chairmen for the 114th congress and extract full list of chairmen names.

In [4]:
chairmen_114 = pd.read_excel('chairmen.xlsx', 'all')
chairmen_114 = chairmen_114[chairmen_114['Congress'] == 114]
chairmen_114 = chairmen_114[(chairmen_114['Chair Committee'].astype(int) == 1) | (chairmen_114['Chair Subcommittee'].astype(int) == 1)] # | (chairmen_114['Vice Chair Committee'].astype(int) == 1) | (chairmen_114['Vice Chair subcommittee'].astype(int) == 1)]

chairmen_names = sorted(chairmen_114['Name'].str.lower().unique())
print(f'There were {len(chairmen_names)} chairmen of Committees and Subcommittees in Congress 114th.')

There were 165 chairmen of Committees and Subcommittees in Congress 114th.


### Get sentences

Extract the data frame of sentences for each document with respective sentence ID and speaker name. Match speaker names extracted from the hearings documents with the complete list of speakers extracted from the bills data frame.

In [ ]:
starting_points = []
sentences = []
keywords = ['senator', 'secretary', 'chairman', 'mr.', 'ms.', 'mrs.', 'doc.']

for i, (document_id, document) in enumerate(zip(congressional_hearings_df['document_id'], congressional_hearings_df['document'])):
    sent_text = sent_tokenize(document)
        
    for sentence_id, sentence in enumerate(sent_text):
        sentences.append({
            'document_id': document_id,
            'sentence_id': sentence_id,
            'sentence': sentence,
        })

        sentence_split = sentence.split(' ')
        sentence_length = len(sentence_split) 
        if sentence_length < 2:
            continue
        first_word = sentence_split[0]
        sentence_is_start_sentence = sentence_length == 2 and first_word in keywords
        sentence_is_start_statement = 'statement of'in sentence
        if sentence_is_start_sentence:
            speaker_name = sentence_split[1].strip('.?')
            starting_points.append({
                'document_id': document_id,
                'speaker_name': speaker_name,
                'speech_id': sentence_id,
            })
        elif sentence_is_start_statement:
            try: 
                speaker_name = sent_text[sentence_id+1].split(', ')[0]
                starting_points.append({
                    'document_id': document_id,
                    'speaker_name': speaker_name,
                    'speech_id': sentence_id,
                })
            except IndexError:
                continue

starting_points = pd.DataFrame(starting_points)
sentences = pd.DataFrame(sentences)

sentences_df = pd.merge(sentences, starting_points, left_on=['document_id', 'sentence_id'], \
                     right_on=['document_id', 'speech_id'], how='left')

sentences_df = sentences_df.groupby('document_id').ffill()
sentences_df = sentences_df.fillna(0)

f = lambda x: next(iter(name for name in speaker_names if str(x) in name), 'no match')
sentences_df['clean_speaker_name'] = sentences_df['speaker_name'].apply(f)

sentences_df.drop(columns=['speaker_name'], inplace=True)

print(f'Sentences DF shape: {sentences_df.shape}')
sentences_df.head()

### Get speeches

Group sentences into speeches and extract title, date and chairman name from first block of each document.

In [ ]:
blocks_df = sentences_df.groupby(['document_id', 'speech_id'])['sentence'].apply(lambda x: ','.join(x)).reset_index()
blocks_df.columns = ['document_id', 'speech_id', 'speech']

document_title = []
hearing_date = []

for i, (speech_id, speech) in enumerate(zip(blocks_df['speech_id'], blocks_df['speech'])):
    if speech_id == 0:
        title = speech.split('\n')[1].replace(' - ', '')
        regex_date = re.compile(r'(?:january|february|march|april|may|june|july|august|september|october|november|december)\s+\d{1,2},\s+\d{4}')
        date = regex_date.findall(speech)
        document_title.append(title)
        hearing_date.append(date)
    else:
        document_title.append(np.nan)
        hearing_date.append(np.nan)

chairmen = []

for index, (speech_id, speech) in enumerate(zip(blocks_df['speech_id'], blocks_df['speech'])):
    if speech_id != 0:
        chairmen.append(np.nan)
    elif speech_id == 0:
        split_speech = speech.splitlines()
        chairman_names = []
        for i, line in enumerate(split_speech):
            try:
                line_plus_2_long = split_speech[i+2] + split_speech[i+3] + split_speech[i+4] + split_speech[i+5]
                
                if 'committee on' in line:
                        
                    if ' chairman' in line_plus_2_long:
                        chairman_name = line_plus_2_long.split(',')[0].lstrip() 
                        chairman_names.append(chairman_name)
                        
                    if ' chair' in line_plus_2_long:
                        chairman_name = line_plus_2_long.split(',')[0].lstrip() 
                        chairman_names.append(chairman_name)
                        
                    if '(chairman)' in line_plus_2_long:
                        line_plus_2_long = line_plus_2_long.split('(chairman)')[0]
                        chairman_name = line_plus_2_long.split('hon.,')[-1].lstrip() 
                        chairman_names.append(chairman_name)
                        
                    if '(chairwoman)' in line_plus_2_long:
                        line_plus_2_long = line_plus_2_long.split('(chairwoman)')[0]
                        chairman_name = line_plus_2_long.split('hon.,')[-1].lstrip() 
                        chairman_names.append(chairman_name)
                        
                    else:
                        continue
                
                if 'first session' in line:
                    
                    if ' chairman' in line_plus_2_long:
                        chairman_name = line_plus_2_long.split(',')[0].lstrip() 
                        chairman_names.append(chairman_name)
                    else:
                        continue
                        
                if 'second session' in line:
                    
                    if ' chairman' in line_plus_2_long:
                        chairman_name = line_plus_2_long.split(',')[0].lstrip() 
                        chairman_names.append(chairman_name)
                    else:
                        continue
                        
                elif '              house                                 senate' in line:
                    
                    if 'chairman' in line_plus_2_long:
                        chairman_name = line_plus_2_long.split(', ')[0].lstrip()
                        chairman_names.append(chairman_name)
                    else:
                        continue
                    
                else:
                    continue
                        
            except IndexError:
                continue
                        
        chairmen.append(chairman_names)
        
blocks_df['document_title'] = document_title

blocks_df['hearing_date'] = hearing_date
blocks_df['hearing_date'] = blocks_df['hearing_date'].str[0]

blocks_df['chairman'] = chairmen
blocks_df['chairman'] = blocks_df['chairman'].str[0]

chairman_dict = {
                 'jerry moran ' : 'jerry moran',
                 'john thune' : 'john r. thune',
                 'lamar alexander ' : 'lamar alexander',
                 'tom cole' : 'thomas cole',
                 'richard c. shelby ' : 'richard c. shelby', 
                 'lindsey graham' : 'lindsey o. graham',
                 'lindsey graham ': 'lindsey o. graham', 
                 'roy blunt ': 'roy blunt',
                 'susan collins, ': 'susan collins', 
                 'john hoeven ': 'john hoeven',
                 'enzi' : 'michael b. enzi', 
                 'one hundred fourteenth congress                              ----------                                                 david vitter' : 'david vitter',
                 '----------                                                 david vitter' : 'david vitter',
                 'one hundred fourteenth congress                             first session                  james m. inhofe' : 'james m. inhofe',
                 'one hundred fourteenth congress                             second session                  james m. inhofe' : 'james m. inhofe',
                 'one hundred fourteenth congress                             first session                  barbara boxer': 'barbara boxer',
                 'subcommittee on energy and water development                   michael k. simpson' : 'michael k. simpson',
                 'related agencies appropriations                mario diaz-balart' : 'mario diaz-balart',
                 'subcommittee on defense              rodney p. frelinghuysen': 'rodney p. frelinghuysen',
                 'subcommittee on homeland security                     john r. carter': 'john r. carter',
                 'subcommittee on defense       rodney p. frelinghuysen': 'rodney p. frelinghuysen',
                 'subcommittee on financial services and general government                    ander crenshaw' : 'ander crenshaw',
                 'subcommittee on energy and water development                                                                       michael k. simpson': 'michael k. simpson',
                 'related agencies                       ken calvert' : 'ken calvert',
                 "william m. ``mac'' thornberry" : 'william m. thornberry',
                 "one hundred fourteenth congress             william m. ``mac'' thornberry": 'william m. thornberry',
                 'subcommittee on interior': 'ken calvert', 
                 'hon.': 'lamar s. smith', 
                 'chairman                                  co-chairman        alcee l. hastings': 'christopher h. smith',
                 'alcee l. hastings': 'christopher h. smith',
                 'subcommittee on commerce': 'john abney culberson', 
                 'drug administration': 'robert b. aderholt', 
                 '[established by s. res.': 'richard burr', 
                 'washington': 'jeff flake'
               }

blocks_df= blocks_df.replace({'chairman': chairman_dict})

blocks_df['speech_len'] = blocks_df['speech'].str.split().apply(len)

blocks_df = blocks_df.groupby('document_id').ffill()

print(f'Blocks DF shape: {blocks_df.shape}')
blocks_df.head()

In [ ]:
speeches_df = pd.merge(blocks_df, sentences_df, left_on=['document_id', 'speech_id'], \
                     right_on=['document_id', 'speech_id'], how='left')

speeches_df = speeches_df[~speeches_df['clean_speaker_name'].isin(['no match'])]
speeches_df['sentence_len'] = speeches_df['sentence'].str.split().apply(len)
speeches_df = speeches_df[speeches_df['sentence_len'] > 3]

speeches_df.to_csv('speeches_df.csv', index=False)

print(f'Speeches DF shape: {speeches_df.shape}')
speeches_df.head()

### Get paragraphs

Break speeches into paragraphs of consistent length.

In [ ]:
TOO_SMALL_LENGTH = 100
TOO_LARGE_LENGTH = 200

def split_paragraph(paragraph):
    import copy
    list_of_paragraphs = []
    remaining_paragraph = copy.copy(paragraph)
    paragraph_split = remaining_paragraph.split('. ')
    i = 0
    while len(remaining_paragraph.split()) > TOO_LARGE_LENGTH:
        paragraph_piece = ''
        while len(paragraph_piece.split()) < TOO_SMALL_LENGTH:
            paragraph_piece += paragraph_split[i]
            i += 1
            remaining_paragraph = '. '.join(paragraph_split[i:])
            
        list_of_paragraphs.append(paragraph_piece)
    
    if len(remaining_paragraph.split()) > TOO_SMALL_LENGTH:
        list_of_paragraphs.append(remaining_paragraph)
        
    return list_of_paragraphs

characters = ["-", "...", "''", "``", "@", "#",  
              '--', '=', '_', '..', '|', "/",
              '~', '—', '•', '“', '–', '>', '*']

def clean_text(text):
    '''Remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = text.replace(' \n', ' ')
    for character in characters:
        text = text.replace(character, " ")
    text = ' '.join(text.split())
    return text

In [ ]:
skip_iterations = []
paragraphs = []

for index, speech in zip(blocks_df['speech_id'], blocks_df['speech']):
    paragraph_text = re.split('[.?!]\n    ', speech)
    for i, paragraph in enumerate(paragraph_text):
        if i in skip_iterations:
            continue
            
        paragraph_length = len(paragraph.split())
        paragraph_too_small = paragraph_length < TOO_SMALL_LENGTH
        paragraph_too_large = paragraph_length > TOO_LARGE_LENGTH
        if paragraph_too_small:
            try:
                i_plus = 1
                while len(paragraph.split()) < TOO_SMALL_LENGTH:
                    paragraph += paragraph_text[i+i_plus]
                    if len(paragraph.split()) > TOO_LARGE_LENGTH:
                        raise IndexError
                    skip_iterations.append(i+i_plus)                    
                    i_plus += 1
                    
            except IndexError:
                continue
                
        elif paragraph_too_large:
            smaller_paragraphs = split_paragraph(paragraph)
            for p in smaller_paragraphs:
                paragraphs.append({
                    'speech_id': index,
                    'paragraph': re.sub(' +', ' ', p),
                })
                
            continue
        
        if len(paragraph.split()) < TOO_SMALL_LENGTH:
            break
            assert False
        paragraphs.append({
            'speech_id': index,
            'paragraph': re.sub(' +', ' ', paragraph),
        })
            
            
paragraphs_df = pd.DataFrame(paragraphs).reset_index().rename(columns={'index': 'paragraph_id'})

paragraphs_df['paragraph_len'] = paragraphs_df['paragraph'].str.split().apply(len)
paragraphs_df['paragraph'] = [clean_text(paragraph) for paragraph in paragraphs_df['paragraph']]

paragraphs_df = paragraphs_df[paragraphs_df['paragraph_len'] <= TOO_LARGE_LENGTH]

print(f'Paragraphs DF shape: {paragraphs_df.shape}')

paragraphs_df.head()

In [ ]:
_paragraphs_blocks_df = pd.merge(paragraphs_df, blocks_df, on='speech_id', how='left')
_paragraphs_temp_df = pd.merge(_paragraphs_blocks_df, speeches_df, on='document_id', how='left')

congress_114_paragraphs_df = pd.merge(_paragraphs_temp_df, congressional_hearings_df.reset_index().drop(columns=['document']), left_on='document_id', right_on='index', how='left')

congress_114_paragraphs_df.drop(columns=['index'], inplace=True)

print(f'Full 114th Congress Paragraphs DF shape: {congress_114_paragraphs_df.shape}')
congress_114_paragraphs_df.head()

In [ ]:
congress_114_paragraphs_df.document_id.nunique()

In [ ]:
speaker_paragraphs_df = congress_114_paragraphs_df[congress_114_paragraphs_df['speaker'].isin(chairmen)]
chairman_paragraphs_df = congress_114_paragraphs_df[congress_114_paragraphs_df['chairman'] == congress_114_paragraphs_df['speaker']]

def FormatParagraph(paragraph):
    punc_filter = re.compile('([.!?]\s*)')
    split_with_punctuation = punc_filter.split(paragraph)
    final = ''.join([i.capitalize() for i in split_with_punctuation])+'.'
    return final

percentage_of_speech = []
for i, row in speaker_paragraphs_df.iterrows():
    _percentage_of_speech = row['paragraph_len'] / speaker_paragraphs_df.groupby('speaker')['paragraph_len'].sum()[row['speaker']] * 100
    _percentage_of_speech = round(_percentage_of_speech, 2)
    percentage_of_speech.append(_percentage_of_speech)

speaker_paragraphs_df['paragraph'] = [FormatParagraph(paragraph) for paragraph in speaker_paragraphs_df['paragraph']]
speaker_paragraphs_df['percentage_of_speech'] = percentage_of_speech
speaker_paragraphs_df.to_excel('speaker_paragraphs_df.xlsx', index=False)
print(f'114th Congress Speaker Paragraphs DF shape: {speaker_paragraphs_df.shape}')
speaker_paragraphs_df.head()

In [ ]:
print(f'There are {speaker_paragraphs_df.speaker.nunique()} speakers represented in the data')
speaker_paragraphs_df.speaker.value_counts()